# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [4]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170500096/170498071 [==============================] - 37s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [5]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [6]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [9]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    device = tf.test.gpu_device_name()
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [10]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
        self.conv1 = tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same")
        self.conv2 = tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same")
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes, activation='softmax')
        
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc1(x)
        
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [11]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [28]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    
    #print_every = 100
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [16]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.011246681213379, Accuracy: 9.375, Val Loss: 2.882659912109375, Val Accuracy: 10.0
Iteration 100, Epoch 1, Loss: 2.258265256881714, Accuracy: 27.8929443359375, Val Loss: 1.925319790840149, Val Accuracy: 38.29999923706055
Iteration 200, Epoch 1, Loss: 2.0924949645996094, Accuracy: 31.91075897216797, Val Loss: 1.8935034275054932, Val Accuracy: 39.79999923706055
Iteration 300, Epoch 1, Loss: 2.0109057426452637, Accuracy: 33.83513259887695, Val Loss: 1.8911669254302979, Val Accuracy: 36.70000076293945
Iteration 400, Epoch 1, Loss: 1.9419808387756348, Accuracy: 35.586814880371094, Val Loss: 1.750265121459961, Val Accuracy: 40.70000076293945
Iteration 500, Epoch 1, Loss: 1.8955000638961792, Accuracy: 36.77021026611328, Val Loss: 1.6563971042633057, Val Accuracy: 43.0
Iteration 600, Epoch 1, Loss: 1.8634766340255737, Accuracy: 37.73658752441406, Val Loss: 1.689975380897522, Val Accuracy: 41.80000305175781
Iteration 700, Epoch 1, Loss: 1.8370387554168701, Accuracy:

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [17]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, 
                                        momentum=0.9, 
                                        nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.336851119995117, Accuracy: 6.25, Val Loss: 2.3382980823516846, Val Accuracy: 9.09999942779541
Iteration 100, Epoch 1, Loss: 1.9156773090362549, Accuracy: 31.52846336364746, Val Loss: 1.678661584854126, Val Accuracy: 40.70000076293945
Iteration 200, Epoch 1, Loss: 1.7539393901824951, Accuracy: 37.82649230957031, Val Loss: 1.4661924839019775, Val Accuracy: 49.39999771118164
Iteration 300, Epoch 1, Loss: 1.6599301099777222, Accuracy: 41.19082260131836, Val Loss: 1.403286337852478, Val Accuracy: 49.70000076293945
Iteration 400, Epoch 1, Loss: 1.5858314037322998, Accuracy: 43.835723876953125, Val Loss: 1.3368498086929321, Val Accuracy: 54.29999923706055
Iteration 500, Epoch 1, Loss: 1.5354046821594238, Accuracy: 45.42165756225586, Val Loss: 1.308249592781067, Val Accuracy: 54.70000076293945
Iteration 600, Epoch 1, Loss: 1.505013346672058, Accuracy: 46.55261993408203, Val Loss: 1.3033357858657837, Val Accuracy: 55.5
Iteration 700, Epoch 1, Loss: 1.47608304023742

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [18]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.003453493118286, Accuracy: 20.3125, Val Loss: 2.788337230682373, Val Accuracy: 13.899999618530273
Iteration 100, Epoch 1, Loss: 2.248304843902588, Accuracy: 28.62004852294922, Val Loss: 1.9096227884292603, Val Accuracy: 38.20000076293945
Iteration 200, Epoch 1, Loss: 2.0852718353271484, Accuracy: 32.291664123535156, Val Loss: 1.8962085247039795, Val Accuracy: 39.0
Iteration 300, Epoch 1, Loss: 2.0041298866271973, Accuracy: 34.36461639404297, Val Loss: 1.8695108890533447, Val Accuracy: 36.5
Iteration 400, Epoch 1, Loss: 1.9322339296340942, Accuracy: 36.23363494873047, Val Loss: 1.705717921257019, Val Accuracy: 43.39999771118164
Iteration 500, Epoch 1, Loss: 1.886417269706726, Accuracy: 37.259857177734375, Val Loss: 1.6547598838806152, Val Accuracy: 43.599998474121094
Iteration 600, Epoch 1, Loss: 1.8547013998031616, Accuracy: 38.15515899658203, Val Loss: 1.677112340927124, Val Accuracy: 42.099998474121094
Iteration 700, Epoch 1, Loss: 1.8300271034240723, Ac

Альтернативный менее гибкий способ обучения:

In [19]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

Train on 49000 samples, validate on 1000 samples
49000/49000 [==============================] - 196s 4ms/sample - loss: 1.8165 - sparse_categorical_accuracy: 0.3883 - val_loss: 1.6107 - val_sparse_categorical_accuracy: 0.4530
10000/1 [=============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.646543049621582, 0.4315]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [22]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu",
                               padding="same",
                               input_shape=input_shape),
        
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", 
                               padding="same"),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    
    model = tf.keras.Sequential(layers)
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, 
                                        momentum=0.9, 
                                        nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.4049108028411865, Accuracy: 7.8125, Val Loss: 2.383795976638794, Val Accuracy: 8.0
Iteration 100, Epoch 1, Loss: 2.1008002758026123, Accuracy: 24.50495147705078, Val Loss: 1.9476102590560913, Val Accuracy: 32.0
Iteration 200, Epoch 1, Loss: 1.9960263967514038, Accuracy: 29.065608978271484, Val Loss: 1.8286945819854736, Val Accuracy: 38.5
Iteration 300, Epoch 1, Loss: 1.9322909116744995, Accuracy: 31.79505729675293, Val Loss: 1.760584831237793, Val Accuracy: 41.5
Iteration 400, Epoch 1, Loss: 1.871129035949707, Accuracy: 34.219139099121094, Val Loss: 1.6992824077606201, Val Accuracy: 43.0
Iteration 500, Epoch 1, Loss: 1.8266141414642334, Accuracy: 35.80963134765625, Val Loss: 1.6380729675292969, Val Accuracy: 44.80000305175781
Iteration 600, Epoch 1, Loss: 1.7938010692596436, Accuracy: 37.081424713134766, Val Loss: 1.605829119682312, Val Accuracy: 45.10000228881836
Iteration 700, Epoch 1, Loss: 1.764315128326416, Accuracy: 38.144168853759766, Val Loss: 1.55

In [23]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

Train on 49000 samples, validate on 1000 samples
49000/49000 [==============================] - 182s 4ms/sample - loss: 1.6286 - sparse_categorical_accuracy: 0.4221 - val_loss: 1.3874 - val_sparse_categorical_accuracy: 0.5100
10000/1 [=============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.4225532176971436, 0.4916]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [24]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [25]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.906797409057617, Accuracy: 14.0625, Val Loss: 2.7837460041046143, Val Accuracy: 12.100000381469727
Iteration 100, Epoch 1, Loss: 2.215958833694458, Accuracy: 29.006805419921875, Val Loss: 1.889789342880249, Val Accuracy: 38.80000305175781
Iteration 200, Epoch 1, Loss: 2.067660093307495, Accuracy: 32.26057434082031, Val Loss: 1.8260499238967896, Val Accuracy: 41.400001525878906
Iteration 300, Epoch 1, Loss: 1.9941507577896118, Accuracy: 34.219268798828125, Val Loss: 1.8828232288360596, Val Accuracy: 37.400001525878906
Iteration 400, Epoch 1, Loss: 1.9274617433547974, Accuracy: 36.05439758300781, Val Loss: 1.7160091400146484, Val Accuracy: 43.5
Iteration 500, Epoch 1, Loss: 1.8837628364562988, Accuracy: 37.05401611328125, Val Loss: 1.648506999015808, Val Accuracy: 44.10000228881836
Iteration 600, Epoch 1, Loss: 1.8538209199905396, Accuracy: 37.9627685546875, Val Loss: 1.7052292823791504, Val Accuracy: 41.900001525878906
Iteration 700, Epoch 1, Loss: 1.829020

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [38]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

       
        self.conv1 = tf.keras.layers.Conv2D(48, (5, 5), padding="same", activation='relu')
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        self.conv2 = tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation='relu')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(512, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(10, activation='softmax')
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.pool2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dense3(x)
        

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.362255811691284, Accuracy: 15.625, Val Loss: 2.3469388484954834, Val Accuracy: 11.90000057220459
Iteration 700, Epoch 1, Loss: 1.427430272102356, Accuracy: 49.135162353515625, Val Loss: 1.1883763074874878, Val Accuracy: 59.500003814697266
Iteration 1400, Epoch 2, Loss: 1.0547244548797607, Accuracy: 62.871559143066406, Val Loss: 1.0329102277755737, Val Accuracy: 64.4000015258789
Iteration 2100, Epoch 3, Loss: 0.8834841847419739, Accuracy: 68.90377807617188, Val Loss: 1.00808584690094, Val Accuracy: 65.80000305175781
Iteration 2800, Epoch 4, Loss: 0.754814088344574, Accuracy: 73.37537384033203, Val Loss: 0.9844765067100525, Val Accuracy: 68.19999694824219
Iteration 3500, Epoch 5, Loss: 0.6524689793586731, Accuracy: 77.10955047607422, Val Loss: 1.1002081632614136, Val Accuracy: 69.0
Iteration 4200, Epoch 6, Loss: 0.5720471739768982, Accuracy: 80.00337219238281, Val Loss: 1.0267812013626099, Val Accuracy: 70.0
Iteration 4900, Epoch 7, Loss: 0.5129932165145874,

Опишите все эксперименты, результаты. Сделайте выводы.

1) я пробовал сделать трёхслойную архитектуру которая была до этого, только добавил после каждой свёртки слой MaxPooling и BatchNormalization и получился довольно плохой результат (~50-55%)


2) два блока: Свёртка -> MaxPooling,затем Flatten слой и после него полносвязный слой из 512 нейронов, потом Dropout-слой (параметр вероятности 0.2), после которого снова полносвязный слой (64 нейронка) и затем уже возвращаются 10 нейронов на последнем слое. Точность на валидационной выборке получилась 69-70%. Впринципе хорошо, но как-то неуверенно..

3) всё также два блока Свёртка -> MaxPooling, но в этот раз количество фильтров 64 и 16, pool_size = (3,3), полносвязный слой теперь 128 нейронов, а вероятность в dropout = 0.5. Точность получилась 65-67%

4) пришла мысль сделать модель глубже. Добавил третий блок Свёртка -> MaxPooling (количество фильтров 64 16 6, размеры ядер (5,5) (3,3) (2,2)), полносвязный слой 128 нейронов, dropout=0.5. Точность снова в районе 65%.

5) Думаю стоит вернуться к той архитектуре когда получалось 69-70% точности и увеличить число фильтров на первом свёрточном слое до 48. Точность получилась 70.2 %